## Jupyter Notebook Example - Irradiance

This jupyter notebook is very similar to the Irradiance notebook, but takes the output from the looping workflow example.
This particular examples shows the expected power output from an real-world example, the STFC Avon Rd. Carpark.

<img src="https://www.ukri.org/wp-content/uploads/2022/07/STFC-120722-RutherfordAppletonLaboratory.jpg" alt="alt text" width="50%"/>

In [ ]:
import os
import glob
from pathlib import Path
import pandas
import json
import math
from matplotlib import pyplot

mdp = os.path.join(Path.home(), 'data')
chkfldr = os.path.join(mdp, os.listdir(mdp)[0])

def getdataframe(fname):
    if os.path.isfile(fname):
        jsonpath = fname
    else:
        jsonpath = os.path.join(mdp, os.listdir(mdp)[0], fname)
    print("Loading", jsonpath)
    datf = pandas.read_json(jsonpath)
    return datf

# Load in the Dataframes for the various PV array configurations
dfs = []
dfldrcont = glob.glob(os.path.join(chkfldr, "*.json"))
for i in range(4):
    filenam = dfldrcont[i]
    tdf = getdataframe(filenam)
    dfs.append(tdf)
# Now scale (by area) and sum the datafiles Azim, 0: 70 deg., 1: 250 , 2: 150, 3: 330
# Note this is just a visual estimate of the surface area.
ar = [0.0]*4
cos8 = math.cos(8.0/180.0*math.pi)
ar[0] = 0.5*(3.0*86.9*11.0 + 2.0*67.6*11.0 + 40*9.5 + 79.2*11.0)/cos8
ar[1] = ar[0] + (100.0*5.5)/cos8
ar[2] = (84.0*5.5 + 28.0*5.5)/cos8
print ("Area of each type in m2", ar)
for i in range(4):
    dfs[i]['POA'] = dfs[i]['POA']*ar[i] * 1.0e-6

In [ ]:
# Note, this will need a years worth of data to properly display graphs
pyplot.rcParams['figure.figsize'] = [10, 6]
import datetime
from matplotlib import pyplot
df =  dfs[0]+dfs[1]+dfs[2]
pyplot.rcParams['figure.figsize'] = [10, 8]
fig, axes = pyplot.subplots(nrows=2, ncols=2)
df.plot(ax=axes[0,0])
axes[0,0].set_xlim([datetime.date(2023, 3, 20), datetime.date(2023, 3, 21)])
#axes[0,0].set_ylim(0.0, 950.0)
axes[0,0].set_ylabel("Power ($MW$)")
axes[0,0].get_legend().set_visible(False)
axes[0,0].set_title("Spring Equinox")
df.plot(ax=axes[0,1])
axes[0,1].set_xlim([datetime.date(2023, 6, 21), datetime.date(2023, 6, 22)])
#axes[0,1].set_ylim(0.0, 950.0)
axes[0,1].set_title("Summer Solstice")
df.plot(ax=axes[1,0])
axes[1,0].set_xlim([datetime.date(2023, 9, 22), datetime.date(2023, 9, 23)])
#axes[1,0].set_ylim(0.0, 950.0)
axes[1,0].set_ylabel("Power ($MW$)")
axes[1,0].set_title("Autumn Equinox")
df.plot(ax=axes[1,1])
axes[1,1].set_xlim([datetime.date(2023, 12, 21), datetime.date(2023, 12, 22)])
#axes[1,1].set_ylim(0.0, 950.0)
axes[1,1].set_title("Winter Solstice")

In [ ]:
pandas.options.mode.chained_assignment = None  # default='warn' , get rid of the false-pos warning
dfml = []
mn = []
mnu = []
mnl = []
mnn = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
imnths = list(range(1,13))
for i in imnths:
    dftmp = df.loc[(df.index.month==i)]
    mn.append(dftmp['POA'].mean())
    mnu.append(dftmp['POA'].max())
    # Get min
    rsmp = dftmp.resample('D')['POA']
    newmdf = rsmp.transform('max')
    dftmp['MIN'] = rsmp.transform('max')
    mnl.append(dftmp['MIN'].min())
    dfml.append(dftmp)
pyplot.rcParams['figure.figsize'] = [6, 4]
fig = pyplot.figure()
ax = fig.add_subplot(1, 1, 1)
ax.plot(mnn, mn, 'o', color='brown')
ax.plot(mnn, mnu, color='orange')
ax.plot(mnn, mnl, color='orange')
ax.fill_between(mnn, mnu, mnl, color='palegoldenrod')
rating = 1.62 # MW
ax.axhline(y=rating, color='green', linestyle='--')
rating_adj = 1.62/0.285 # ~29% Efficiency Assumed here
ax.axhline(y=rating_adj, color='green', linestyle='-')
ax.grid()
#ax.legend().set_visible(True)
ax.set_title("Mean Power+Range for Avon Road Site (STFC-RAL)")
ax.set_ylabel("Power ($MW$)")
pandas.options.mode.chained_assignment = 'warn'  # default='warn' , switch back